# 0. Google Drive Mount

In [ ]:
from google.colab import drive
!drive.mount('/content/drive/suyun_speech_corpus')

/bin/bash: -c: line 1: syntax error near unexpected token `'/content/drive/suyun_speech_corpus''
/bin/bash: -c: line 1: `drive.mount('/content/drive/suyun_speech_corpus')'


# 1. Settings

In [ ]:
# Install selenium
!pip install selenium

In [ ]:
# Confirm Selenium
import selenium
print(selenium.__version__)

4.22.0


In [ ]:
# Install chromedriver : Only once

#!apt-get update
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/suyun_speech_corpus'

In [ ]:
# Install autoinstaller : to match the version
!pip install chromedriver_autoinstaller

In [ ]:
# selenium으로 키를 조작하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ETC
import time # to wait not to seem as "bot"
import sys
import chromedriver_autoinstaller
import csv

In [ ]:
chrome_path = "/content/drive/MyDrive/suyun_speech_corpus/chromedriver"

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

# 3. Crawling - 대통령실
https://www.president.go.kr/president/speeches

In [ ]:
# Helper function to click page button
def click_section_button():
    try:
        section_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[2]/button[3]')))
        driver.execute_script("arguments[0].scrollIntoView();", section_button) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", section_button)  # Click with JavaScript
    except TimeoutException:
        print(f"Section button not found")
        raise  # if TimeoutException : throw Exception & quit

In [ ]:
# Helper function to click page button
def click_page_button(page_num):
    rest = page_num%5
    if rest== 0:
        num = 5
    else:
        num = rest
    try:
        page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[2]/ul/li[{num}]/button')))
        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
    except TimeoutException:
        print(f"Page button {page_num} not found")
        raise  # if TimeoutException : throw Exception & quit

In [ ]:
def crawl_speech():
    global idx
    global speech_data
    global driver
    try:
        o_title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/article/div[2]/div/dl/dt/div[1]/p')))
        o_subtitle = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div[2]/div/dl/dd/div/div[2]/p')
        o_date = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div[2]/div/dl/dt/div[2]/span')
        #o_speech = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div[2]/div/dl/dd/div/div[3]')
        o_speech = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
            EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/article/div[2]/div/dl/dd/div/div[3]')))
        driver.execute_script("arguments[0].scrollIntoView();", o_speech)
        time.sleep(1)
        title = o_title.text
        subtitle = o_subtitle.text
        date = o_date.text
        speech = o_speech.text
        speaker = "윤석열"

        d = {"index": idx, "title": title, "subtitle": subtitle, "date": date, "speaker": speaker, "speech": speech}
        speech_data.append(d)
        print(f"{idx}. {title}")
        idx += 1
    except NoSuchElementException as e:
        print(f"An element was not found: {e}")
    except TimeoutException as e:
        print(f"An element load timed out: {e}")

In [ ]:
# Run Chrome Driver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
speech_data = []
idx = 1

# WebDriver Reset
driver.get('https://www.president.go.kr/president/speeches')
time.sleep(2)


for page_num in range(1,41):
    click_page_button(page_num)
    for title_num in range(1, 6):  # 예제에서는 첫 번째 페이지의 두 개 연설을 크롤링
        if (page_num == 1) & (title_num == 1):
            first_click = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div/div[1]/div[2]/a/div[2]/p[1]/span')
            first_click.click()
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        try:
            rest_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[1]/ul/li[{title_num}]/div/a/p[1]')))
            driver.execute_script("arguments[0].scrollIntoView();", rest_click)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", rest_click)
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"An Error Occured: {str(e)}")
            break
    if page_num%5 ==0:
        click_section_button()
driver.quit()

print("크롤링 완료!")

1. "국정의 목표는 하나, 민생의 어려움을 해결하고 국민을 더 행복하게 만드는 것"
2. "국민 모두가 행복하고 마음이 건강한 사회가 될 수 있도록 지원할 것"
3. "나라를 지키기 위해 고귀한 청춘을 바친 영웅들의 희생과 헌신에 보답하기 위해 자유 대한민국 굳건히 지킬 것"
4. "고귀한 생명 바쳐 지킨 대한민국의 자유와 평화, 글로벌 중추국가로 도약 시킬 것"
5. "저출생, 인구절벽, 지방소멸과 같은 위기 극복을 위해 바꿀 수 있는 것은 모두 바꾼다는 절박함으로 힘 모아야"
6. "대한민국의 오늘을 만든 주역 경북의 성공적인 지방시대 위해 적극 지원할 것"
7. "인구전략기획부 신설, 양립, 양육, 주거 3대 핵심 분야 역량 집중해 저출생 문제 해결 매진할 것"
8. "정부는 보다 많은 기업들이 중앙아시아와 교류할 수 있도록 '기회의 K실크로드'를 개척해 나갈 것"
9. "어려울 때 서로 돕는 진정한 친구 우즈벡과 함께 미래로 나아갈 것"
10. "고려인 동포들을 따뜻하게 품어준 형제 국가 우즈베키스탄과 함께 미래로 도약해 나갈 것"
11. "한국은 우즈베키스탄의 부존자원과 발전에 대한 열망을 접목시켜 양국의 공동 발전을 창출해 낼 것"
12. "동포 여러분과 긴밀히 소통하면서 중앙아 국가들과의 파트너십을 더 강화해 나갈 것"
13. "청년창업을 지원하는 것이 현대국가의 중요한 과제"
14. "한국과 카자흐스탄은 상호 보완적 산업구조를 보유한 최적의 파트너, 경제 협력의 지평 넓혀 나가야"
15. "중앙아의 심장이자 경제 허브인 카자흐스탄과 함께 밝고 풍요로운 미래를 창조해 나갈 것"
16. "핵심 파트너이자 전략적 동반자인 카자흐스탄과 함께 손잡고 더욱 번영하는 미래로"
17. "카자흐스탄 등 중앙아시아는 중요 협력 대상, 동포 여러분들의 땀과 노력이 협력 발전에 큰 역할"
18. "투르크메니스탄의 명마 아할 테케처럼 양국 경제 협력의 속도를 더욱 높여야"
19. "협력의 지평 확대, 호혜적 동반자 관계를 더욱 탄탄하게 만들 것"
20. "국가를 위해

In [ ]:
idx = 201

# WebDriver Reset
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.president.go.kr/president/speeches')
time.sleep(2)

for _ in range(8):
  click_section_button()
  time.sleep(1)

for page_num in range(41,71):
    click_page_button(page_num)
    for title_num in range(1, 6):  # 예제에서는 첫 번째 페이지의 두 개 연설을 크롤링
        if (page_num == 1) & (title_num == 1):
            first_click = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div/div[1]/div[2]/a/div[2]/p[1]/span')
            first_click.click()
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        try:
            rest_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[1]/ul/li[{title_num}]/div/a/p[1]')))
            driver.execute_script("arguments[0].scrollIntoView();", rest_click)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", rest_click)
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"An Error Occured: {str(e)}")
            break
    if page_num%5 ==0:
        click_section_button()

print("크롤링 완료!")

201. "정부와 기업은 원팀... 우리 기업의 수출과 수주에 도움되는 일이라면 뛰고 또 뛸 것"
202. "한국과 사우디가 굳건히 다져온 토대 위에 새로운 인프라 경제협력의 시대를 열어가야"
203. "미래기술 파트너십을 기반으로 한국과 사우디가 연대하여 세계의 미래를 열어 나가야"
204. "한국과 사우디는 새로운 분야를 개척하고 이끌어가는 퍼스트 무버로 거듭나야"
205. "사우디와 함께 한 역사가 곧 대한민국 산업발전의 역사...한국과 사우디가 손을 잡으면 강력한 시너지를 만들 것"
206. "의료서비스의 공급과 이용체계를 바로 세우고 지역 필수 인력을 확충하기 위해 현장 의료인, 전문가들과 소통할 것"
207. "국민의 안전 확보라는 책무를 가슴에 새기면서 국민의 경찰로 정진해 주시길"
208. "국민통합은 헌법이라는 규범과 자유·연대 가치를 기제로 이루어 지는 것"
209. "방위산업은 안보와 경제를 뒷받침하는 ‘국가전략산업’"
210. "전국체육대회는 우리나라가 스포츠 강국으로 성장하는 든든한 밑거름"
211. "공생원이 우리 사회의 소외된 이웃들을 돌보는 데 앞장서고, 한일 양국 우정의 상징으로 더욱 발전하길"
212. "대한민국 정부와 국민은 장진호 전투 영웅의 희생을 결코 잊지 않을 것"
213. "청년 화이트해커는 사이버 안보의 중요한 전략자산... 정보보호 산업을 국가 전략 산업으로 육성할 것"
214. "이스라엘-하마스 사태, 국민이 피해를 보거나 위험 발생하지 않도록 철저하게 대비해 주길"
215. "정부는 긴장의 끈을 놓지 않고 민생 어려움이 가중되지 않도록 전력 다해야"
216. "교권은 학생들을 위해서 꼭 있어야 되는 것"
217. "동포들이 조국에 대한 긍지와 자부심을 가질 수 있도록 국제사회에 더 많이 기여하고 협력할 것"
218. "파독 광부, 간호사 여러분의 땀과 헌신을 국가의 이름으로 예우하고 기억할 것"
219. "재향군인회 여러분들께서 안보 의식 강화와 안보태세 확립에 앞장서 주길"
220. "국군 통수권자로서 전투력

In [ ]:
# WebDriver Reset
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.president.go.kr/president/speeches')
time.sleep(2)

for _ in range(14):
  click_section_button()
  time.sleep(1)

In [ ]:
idx = 351

for page_num in range(71,81):
    click_page_button(page_num)
    for title_num in range(1, 6):  # 예제에서는 첫 번째 페이지의 두 개 연설을 크롤링
        if (page_num == 1) & (title_num == 1):
            first_click = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div/div[1]/div[2]/a/div[2]/p[1]/span')
            first_click.click()
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        try:
            rest_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[1]/ul/li[{title_num}]/div/a/p[1]')))
            driver.execute_script("arguments[0].scrollIntoView();", rest_click)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", rest_click)
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"An Error Occured: {str(e)}")
            break
    if page_num%5 ==0:
        click_section_button()

print("크롤링 완료!")

351. "한·일 양국의 미래 협력을 위한 유익한 발걸음이 되길 기대해"
352. "미래의 꿈나무인 어린이들을 위해 더 열심히 노력할 것"
353. "ADB 회원국들이 중심이 돼 국가 간 연대·협력이 새로운 모델을 만들어 나가길 기대"
354. "함께 우리 사회의 변화를 만들어 갑시다"
355. "한·미 동맹 영역은 계속 확장될 것...국민 기회는 더욱 커질 것"
356. "자유를 위협하는 세력에 맞서기 위해서는 힘을 합치고 연대해야"
357. "첨단 산업 클러스터 협력...양국 경제 모두에 큰 시너지가 될 것"
358. "창의적이고 혁신적인 과학기술 협력...한미동맹의 새로운 미래 영역"
359. "확고한 한미연합방위태세 바탕으로 북한 위협에 단호하게 대응해 나갈 것"
360. "문화 영역에는 국가 간 국경 없어...한미 간 문화 협력 강화"
361. "대한민국 국민을 대표해 깊은 감사와 무한한 경의를 표해"
362. "한미 양국 미래세대는 또 다른 70년을 이어갈 한미동맹으로부터 무한한 혜택을 받을 것"
363. "한미 글로벌 포괄적 전략동맹의 깊이와 외연을 더욱 확장하고, 미래로 전진해 나갈 것"
364. "한미동맹은 한반도뿐만 아니라 글로벌 평화와 안정에 핵심축이 돼"
365. "한미동맹은 미래로 나아가는 동맹이고 행동하는 동맹"
366. "양국 간 우주동맹이 우주기술·경제 분야에 그치는 것이 아니라, 우주안보 분야로도 확대되기를 바라"
367. "대한민국 정부와 국민은 한국전 참전용사의 희생을 잊지 않고, 한미동맹 굳건히 지켜나갈 것"
368. "명실상부한 첨단 기술 동맹 재확인...새롭고 혁신적인 비즈니스 기회 창출할 수 있길 기대"
369. "한·미는 안정적인 공급망 구축에 최적의 파트너...서로 믿을 수 있기 때문"
370. "한국에 마음껏 투자하고 큰 성공 할 수 있도록 세계 최고 투자 환경 만들 것"
371. "재외동포청 출범으로 동포 여러분과 모국을 긴밀하게 연결해 나갈 것"
372. "이차전지는 반도체와 함께 우리의 안보·전략 자산의 핵심

In [ ]:
idx = 398

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.president.go.kr/president/speeches')
time.sleep(2)

for _ in range(16):
  click_section_button()
  time.sleep(1)

for page_num in range(81,91):
    click_page_button(page_num)
    for title_num in range(1, 6):  # 예제에서는 첫 번째 페이지의 두 개 연설을 크롤링
        if (page_num == 1) & (title_num == 1):
            first_click = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div/div[1]/div[2]/a/div[2]/p[1]/span')
            first_click.click()
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        try:
            rest_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[1]/ul/li[{title_num}]/div/a/p[1]')))
            driver.execute_script("arguments[0].scrollIntoView();", rest_click)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", rest_click)
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"An Error Occured: {str(e)}")
            break
    if page_num%5 ==0:
        click_section_button()
driver.quit()

print("크롤링 완료!")

398. "한일 간 협력의 새 시대를 여는 첫걸음이 되었습니다"
399. "한일관계의 새로운 시대를 열기 위해 함께 노력할 수 있기를 기대합니다"
400. "한일 양국은 더 나은 미래로 나아가기 위한 출발점에 서 있습니다"
401. "국가첨단산업단지 조성 신속 추진...모든 노력 다할 것"
402. "정부는 기업과 원팀...높은 경제 성장·일자리 창출 달성할 수 있도록 노력할 것"
403. "안보는 미래의 핵심축...강력한 해양강군 구축해야"
404. "울산 첨단산업 혁신허브 될 수 있도록 적극 지원할 것"
405. "한-사우디 경제외교 첫발 내딛어...새로운 도약 응원"
406. "당내 선거에는 승자도 패자도 없어...새로 선출될 지도부와 모두 하나 돼야"
407. "진정한 형제 국가임을 보여주는 우정의 상징"
408. "한일 간 미래지향적 협력은 전 세계 자유·평화·번영 지켜줄 것"
409. "대한민국의 부름에 응답한 분들을 정부는 어떠한 경우에도 잊지 않을 것"
410. "기미독립선언의 정신을 계승하여 자유, 평화, 번영의 미래를 만들어 갑시다"
411. "바이오헬스 시장 규모 2,600조 원...제2의 반도체 산업으로 키울 것"
412. “기득권 카르텔 깨고 자유롭고 공정한 시스템 만들 때 혁신 이뤄져”
413. “순방 때 중소벤처기업 동행해 백방으로 뛸 것...한-UAE 투자협력 플랫폼·원스톱 수출 지원단 운영”
414. "변화와 개혁의 길 걸었던 김영삼 대통령 뜻 이어 노동·교육·연금 3대 개혁 추진할 것"
415. "수출·스타트업 활성화로 복합위기 돌파...1호 영업사원으로 최전선에서 뛸 것"
416. "아이들이 커나가는 데 지장 없도록 24시간 소아의료체계 보강할 것"
417. "우주는 경제성장·국가안보 핵심 동력...우주항공청으로 우주경제 시대 열어갈 것"
418. "건설현장 불법행위 엄정 조치...노조 개혁 출발은 노조 회계 투명성 강화"
419. "공공요금·에너지요금·통신비용·금융비용 등 4대 민생 분야 지출 부담 경감할 것"
420. "시

In [ ]:
len(speech_data)

444

In [ ]:
idx = 445

# WebDriver Reset
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.president.go.kr/president/speeches')
time.sleep(2)

for _ in range(18):
  click_section_button()
  time.sleep(1)

for page_num in range(91,101):
    click_page_button(page_num)
    for title_num in range(1, 6):  # 예제에서는 첫 번째 페이지의 두 개 연설을 크롤링
        try:
            rest_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[1]/ul/li[{title_num}]/div/a/p[1]')))
            driver.execute_script("arguments[0].click();", rest_click)
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"An Error Occured: {str(e)}")
            break
    if page_num%5 ==0:
        click_section_button()
driver.quit()

print("크롤링 완료!")

An element load timed out: Message: 
Stacktrace:
#0 0x5bfac4944c7a <unknown>
#1 0x5bfac4627e2c <unknown>
#2 0x5bfac4674661 <unknown>
#3 0x5bfac4674751 <unknown>
#4 0x5bfac46b8f64 <unknown>
#5 0x5bfac46975ed <unknown>
#6 0x5bfac46b6303 <unknown>
#7 0x5bfac4697363 <unknown>
#8 0x5bfac4667247 <unknown>
#9 0x5bfac4667b9e <unknown>
#10 0x5bfac490b24b <unknown>
#11 0x5bfac490f2f1 <unknown>
#12 0x5bfac48f6afe <unknown>
#13 0x5bfac490fe52 <unknown>
#14 0x5bfac48db79f <unknown>
#15 0x5bfac4934638 <unknown>
#16 0x5bfac4934810 <unknown>
#17 0x5bfac4943dac <unknown>
#18 0x7dfee52c8ac3 <unknown>

An element load timed out: Message: 
Stacktrace:
#0 0x5bfac4944c7a <unknown>
#1 0x5bfac4627e2c <unknown>
#2 0x5bfac4674661 <unknown>
#3 0x5bfac4674751 <unknown>
#4 0x5bfac46b8f64 <unknown>
#5 0x5bfac46975ed <unknown>
#6 0x5bfac46b6303 <unknown>
#7 0x5bfac4697363 <unknown>
#8 0x5bfac4667247 <unknown>
#9 0x5bfac4667b9e <unknown>
#10 0x5bfac490b24b <unknown>
#11 0x5bfac490f2f1 <unknown>
#12 0x5bfac48f6afe <un

KeyboardInterrupt: 

In [ ]:
idx = 502

# WebDriver Reset
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.president.go.kr/president/speeches')
time.sleep(2)

for _ in range(16):
  click_section_button()
  time.sleep(1)

for page_num in range(101,110):
    click_page_button(page_num)
    for title_num in range(1, 6):  # 예제에서는 첫 번째 페이지의 두 개 연설을 크롤링
        if (page_num == 1) & (title_num == 1):
            first_click = driver.find_element(By.XPATH, '//*[@id="contents"]/article/div/div[1]/div[2]/a/div[2]/p[1]/span')
            first_click.click()
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        try:
            rest_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="contents"]/article/div/div[1]/ul/li[{title_num}]/div/a/p[1]')))
            driver.execute_script("arguments[0].scrollIntoView();", rest_click)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", rest_click)
            time.sleep(2)
            crawl_speech()
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"An Error Occured: {str(e)}")
            break
    if page_num%5 ==0:
        click_section_button()
driver.quit()

print("크롤링 완료!")

### To CSV

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/President_Office'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/president_office_crawling.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "subtitle", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 444개

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/President_Office/president_office_crawling.csv
